In [1]:
import guppylang
from guppylang import guppy
from guppylang.std.builtins import owned
from guppylang.std.quantum import qubit, measure, h, cx
guppylang.enable_experimental_features()


### Variable not defined

In [2]:
@guppy
def bad(b: bool) -> int:
    if b:
        x = 4
    return x  # x not defined if b is False

bad.compile()  # Raises an error

3 3
3 5


--------------
  | 
3 |     if b:
  |        - ... if this expression is `False`
--------------
Error: Variable not defined (at <In[2]>:5:11)
  | 
3 |     if b:
4 |         x = 4
5 |     return x  # x not defined if b is False
  |            ^ `x` might be undefined ...

Guppy compilation failed due to 1 previous error


Similarly, variables must have a unique type when they are used.

### Different types

In [3]:
@guppy
def bad(b: bool) -> int:
    if b:
        x = 4
    else:
        x = True
    return int(x)  # x has different types depending on b

bad.compile()  # Raises an error

4 4
5 7
6 6
5 7


--------------
  | 
6 |         x = True
  |         - This is of type `bool`
--------------
Error: Different types (at <In[3]>:7:15)
  | 
5 |     else:
6 |         x = True
7 |     return int(x)  # x has different types depending on b
  |                ^ Variable `x` may refer to different types


  Note:
  | 
3 |     if b:
4 |         x = 4
  |         - This is of type `int`

Guppy compilation failed due to 1 previous error


### Copy violation

In [4]:
@guppy
def bad(q: qubit @owned) -> qubit:
    cx(q, q)
    return q

bad.compile()  # Raises an error

3 3
1 3


--------------
  | 
3 |     cx(q, q)
  |        - since it was already borrowed here
--------------
Error: Copy violation (at <In[4]>:3:10)
  | 
1 | @guppy
2 | def bad(q: qubit @owned) -> qubit:
3 |     cx(q, q)
  |           ^ Variable `q` with non-copyable type `qubit` cannot be
  |             borrowed ...

Guppy compilation failed due to 1 previous error


In [5]:
@guppy.declare
def foo(q1: qubit) -> None: ...


@guppy.declare
def use(q: qubit @owned) -> None: ...


@guppy
def test(q: qubit @owned) -> None:
    use(q)


    
    foo(q)


test.compile()


11 11
13 15


--------------
--------------
Error: Copy violation (at <In[5]>:15:8)
   | 
13 | 
14 | 
15 |     foo(q)
   |         ^ Variable `q` with non-copyable type `qubit` cannot be
   |           borrowed ...


  Note:
   | 
10 | def test(q: qubit @owned) -> None:
11 |     use(q)
   |         - since it was already consumed here

Guppy compilation failed due to 1 previous error


In [ ]:
@guppy.struct
class MyStruct:
    q1: qubit
    q2: qubit


@guppy
def foo(b: bool, s: MyStruct @owned) -> MyStruct:
    if b:
        measure(s.q2)


        
    return s


foo.compile()


10 10
12 14


--------------
--------------
Error: Copy violation (at <In[7]>:14:11)
   | 
10 |         measure(s.q2)
   |                 ---- since it was already consumed here
   | 
12 | 
13 | 
14 |     return s
   |            ^ Field `s.q2` with non-copyable type `qubit` cannot be
   |              returned ...

Guppy compilation failed due to 1 previous error


### Drop violation

In [ ]:
@guppy
def bad(q: qubit @owned) -> qubit:
    tmp = qubit()
    cx(tmp, q)
    #discard(tmp)  # Compiler complains if tmp is not explicitly discarded
    return q

bad.compile()  # Raises an error

--------------
--------------
Error: Drop violation (at <In[6]>:3:4)
  | 
1 | @guppy
2 | def bad(q: qubit @owned) -> qubit:
3 |     tmp = qubit()
  |     ^^^ Variable `tmp` with non-droppable type `qubit` is leaked

Help: Make sure that `tmp` is consumed or returned to avoid the leak

Guppy compilation failed due to 1 previous error


In [ ]:
@guppy.struct
class MyStruct:
    q: qubit


@guppy
def foo(b: bool) -> bool:
    s = MyStruct(qubit())
    if b:
        return measure(s.q)
    return False


foo.compile()

### Illegal assignment

In [ ]:
@guppy
def outer(x: int) -> int:
    def nested() -> None:
        x += 1  # Mutation of captured variable x is not allowed
    nested()
    return x

outer.compile_function()  # Raises an error

In [ ]:
@guppy.struct
class MyStruct1:
    x: "MyStruct2"


@guppy.struct
class MyStruct2:
    q: qubit


@guppy.declare
def use(s: MyStruct2 @owned) -> None: ...


@guppy
def foo(s: MyStruct1 @owned) -> qubit:
    use(s.x)
    return s.x.q


foo.compile()


In [ ]:
@guppy
def foo(qs: list[tuple[qubit, bool]] @owned) -> list[qubit]:
    rs: list[qubit] = []
    for q, b in qs:
        rs += [q]
        if b:
            break
    return rs


foo.compile()

In [ ]:
@guppy.struct
class MyStruct:
    q: qubit


@guppy.declare
def use(q: qubit @owned) -> None: ...


@guppy
def test(s: MyStruct, b: bool) -> None:
    if b:
        use(s.q)


test.compile()

13 13
9 11


--------------
--------------
Error: Copy violation (at <In[7]>:11:9)
   | 
 9 | 
10 | @guppy
11 | def test(s: MyStruct, b: bool) -> None:
   |          ^^^^^^^^^^^ Borrowed argument s cannot be returned to the caller ...


  Note:
   | 
11 | def test(s: MyStruct, b: bool) -> None:
12 |     if b:
13 |         use(s.q)
   |             --- since `s.q` with non-copyable type `qubit` was already
   |                 consumed here

Help: Consider writing a value back into `s.q` before returning

Guppy compilation failed due to 1 previous error


In [ ]:



@guppy
def foo(x: int) -> int:
    y = x + 1

    def bar() -> None:

        def baz() -> None:
            y += 2

        baz()

    bar()
    return y

foo.compile_function()


3 3
6 8


--------------
--------------
Error: Illegal assignment (at <In[9]>:8:12)
  | 
3 |     y = x + 1
  |     - `y` defined here
  | 
6 | 
7 |         def baz() -> None:
8 |             y += 2
  |             ^^^^^^ Variable `y` may not be assigned to since `y` is captured
  |                    from an outer scope

Guppy compilation failed due to 1 previous error


In [ ]:
@guppy
def foo(x: int) -> int:
    y = x + 1

    def bar() -> None:
        y += 2

    bar()
    return y

foo.compile_function()

3 3
4 6


--------------
--------------
Error: Illegal assignment (at <In[6]>:6:8)
  | 
3 |     y = x + 1
  |     - `y` defined here
  | 
4 | 
5 |     def bar() -> None:
6 |         y += 2
  |         ^^^^^^ Variable `y` may not be assigned to since `y` is captured
  |                from an outer scope
  Note:

Guppy compilation failed due to 1 previous error
